## Introduction
오늘 강의에서 Dropout regularization이 추가된 Neural networks (dropout neural network)는 variational inference를 이용한 Bayesian neural networks로 해석할 수 있다는 사실을 배웠습니다. 첫 번째 실습에서 dropout neural network의 stochastic output을 이용하여 neural network의 uncertainty를 측정하는 방법에 대해 배웠습니다. 우리가 보통 사용하는 dropout은 각 layer의 모든 (input 또는 hidden) unit들이 같은 dropout rate을 갖도록 합니다. 하지만, 각 layer의 각 unit에 적합한 dropout rate을 일일이 찾는다는 건 불가능에 가깝기 때문에 unit이 각자의 dropout rate을 학습하도록 하면 더 나은 model uncertainty를 얻을 수 있으며 더 좋은 performance도 얻을 수 있을 것입니다. 이번 실습에서는 그것을 가능케 하는 방법 중 하나인 concrete dropout에 대해서 배우고자 합니다.

이번 실습에서 우리는 다음과 같은 내용을 학습할 예정입니다.
* Bayesian inference & Variational inference Review
* Concrete dropout에 사용되는 중요한 개념인 Gumbel-softmax trick을 이용한 Concrete distribution
* Concrete distribution을 통해 각 (input 또는 hidden) unit들의 dropout rate을 학습할 수 있는 concrete dropout
* Concrete dropout을 구성하는 요소들에 대한 이해 및 구현
* 구현한 concrete dropout neural network를 이용하여 각 unit들의 dropout rate 변화 관찰

## Bayesian Inference and Variational Inference (VI)

오전 강의와 앞서 첫 번째 실습에서도 Review했듯이 Bayesian inference의 기본 과정은 다음과 같이 prior를 설정해주고 데이터에 따른 posterior를 구하는 것입니다.

![bayesian](images/bayesian_inference.png)

그런데 $W$가 neural network의 weight parameter인 경우에는 posterior distribution $p(W|\mathcal{D})$를 정확히 계산하기란 불가능합니다. 따라서, variational inference 관점에서는 true posterior를 잘 묘사할 수 있는 또는 계산하기 상대적으로 편한 "variational distribution" $q_{\phi}(W)$를 설정하고 (ex. mean-field Gaussian) true posterior와 variational approximate distribution 사이의 거리를 줄이는 문제로 치환하게 됩니다. 여기서 두 distribution 사이의 거리를 측정할 때는 흔히 알고 있는 KL divergence를 이용합니다. 조금 더 자세히 식을 정리해보면

![elbo](images/elbo.png)

위와 같이 나타낼 수 있습니다. Prior를 우리가 설정해주듯이 variational distribution 또한 우리가 optimize하기 쉬운 형태의 form으로 보통 설정해줍니다. 그래서 mean-field Gaussian distribution이 가장 많이 사용되는 예시입니다. 그럼 이제 dropout neural network가 어떻게 variational inference로 해석되는지 알아봅니다.

## Dropout as a Variational Inference

우리는 오전 강의에서 Dropout regularization이 추가된 Neural network는 Bayesian neural network로 볼 수 있다는 점을 배웠습니다. 또한, 그에 대한 example로 Concrete dropout, Variational dropout, CNN dropout 등을 살펴보았습니다. Dropout은 각각의 (input 또는 hidden) unit들이 Bernoulli distribution을 따르게 됩니다. 다음 그림은 dropout의 intuitive한 설명을 나타냅니다.

![dropout](images/dropout.png)

Dropout이 추가된 Neural network이 어떻게 variational inference로 해석될 수 있는지에 대해 알아보기 전에 Bayesian inference와 variational inference에 대해 review하도록 하겠습니다.

Dropout neural network는 $l^{\text{th}}$ layer의 weight parameter를 $W_l \in \mathbb{R}^{K_{l+1} \times K_l}$이라 했을 때 prior $p(W_l)$는 Gaussian, posterior에 대한 variational distribution은 $q_{M_l}(W_l) = M_l\cdot\mathrm{diag}[\mathrm{Bernoulli}(1-p_l)^{K_l}]$ 를 따르는 것과 동일하다는 것이 알려져 있습니다.

![var_interp](images/interpretation.png)

여기서 variational parameter $\phi = \{M_l, p_l\}$이며 $M_l$은 $l^{\text{th}}$ layer weight의 realization, $p_l$은 각 layer의 dropout rate을 나타냅니다. 자세한 derivation 과정은 생략하도록 하겠습니다. 이를 이용하면 위의 Evidence Lower BOund (ELBO) term에서 KL regularization term은 다음처럼 derive됩니다.

![kl_divergence](images/kl_divergence.png)

여기서 $\mathcal{H}(p_l)$은

![entropy](images/entropy.png)

입니다.

이는 하나의 $l^{\text{th}}$ layer에만 해당되기 때문에 최종적으로 우리가 minimize해야 하는 term은

![elbo2](images/elbo.png)

에서 ELBO이며 우리는 첫 번째 KL term에 대한 derive를 위에서 끝냈습니다. 따라서,

![final_objective](images/final_objective.png)

과 같이 모든 layer에 대한 term을 더해주어야 합니다.

에서 ELBO라고 적힌 Term입니다. ELBO의 첫 번째 Term인 KL divergence는 위에 derive한 식을 그대로 이용하면 되지만 두 번째 Term은 variational distribution을 이용한 sampling이 필요합니다.

하지만, 여기서 문제점은 $p_l$은 discrete한 Bernoulli distribution의 parameter이기 때문에 우리가 일반적으로 사용하는 Backpropagation으로 학습하기 어렵습니다. 따라서, continuous하게 relaxation 해주어야 하며 이를 위해 concrete distribution에 대해 알아보도록 하겠습니다.

## Concrete Distributions (Gumbel-Softmax Trick)

Concrete distribution는 Bernoulli distribution 뿐만 아니라 categorical distribution을 갖는 any random variable에 적용 가능한 테크닉입니다. Categorical probability $(\alpha_1, \alpha_2, \cdots, \alpha_K)$를 갖는 random variable $Z$에 대해 sampling 하는 방법은 다음과 같은 Gumbel distribution 통해 나타낼 수 있습니다.

\begin{align*}
    z = \mathtt{one\_hot} \Big(\arg\max_i [G_i + \log \alpha_i]\Big)
\end{align*}

여기서 $G_i$는 $\mathrm{Gumbel}(0,1)$ distribution을 따릅니다. 하지만, argmax operator가 non-differentiable하기 때문에 class probability $\alpha_i$ 자체를 학습하고 싶은 경우에는 Backpropagation을 하는 데에 여전히 문제가 됩니다 (ex. concrete dropout처럼 dropout rate 자체도 학습하고 싶은 경우). 따라서, argmax operator를 우리가 잘 아는 softmax로 relaxation을 하면 문제는 해결됩니다. 이를 그림으로 나타내면 다음과 같습니다.

![gumbel_softmax](images/gumbel_softmax.png)

여기서 $\lambda$는 continuous relaxation의 정도를 나타내는 parameter이며 이 값이 커질수록 approximate된 distribution은 조금 더 smooth한 형태를 띄게 됩니다. 다음은 조금 더 이해를 돕기 위한 그림입니다. 

![concrete_distributions](images/concrete_distributions.png)

특수한 case로 Bernoulli random variable $Z$에 대해서는 다음과 같은 concrete distribution trick을 이용하여 sampling이 가능합니다.

\begin{align}
    z = \mathrm{Sigmoid}\Big((\log u - \log (1 - u) + \log p - \log (1 - p)) / \lambda\Big)
\end{align}

여기서 $u$는 $\mathrm{Uniform}(0,1)$ random variable입니다. tf.random.uniform과 같은 함수로 쉽게 sampling이 가능합니다. 따라서, 원래 dropout rate $p$ 자체를 sampling하는 작업이 uniform random variable $U$를 대신 sampling하는 것으로 치환됨으로써 dropout $p$ 역시 backpropagation을 이용하여 학습을 할 수 있습니다.

조금 더 직관적인 설명을 위해 우리가 보통 variational distribution $q_{\phi}(z)$를 $\mathcal{N}(z|\mu, \sigma)$로 mean-field Gaussian으로 설정하는 경우를 생각해봅시다. Sampling 과정에서 Variational parameter $(\mu, \sigma)$가 직접적으로 연관이 되어 있기 때문에 $(\mu, \sigma)$를 학습하기 위해서 reparametrization trick을 이용한다는 것을 배웠습니다 (ref. Variational Autoencoder (VAE)).

![reparam](images/reparam.png)

Gumbel-softmax trick도 같은 맥락으로 이해할 수 있습니다. 그림으로 표현하면

![gumbel_desc](images/gumbel_description.png)

로 나타낼 수 있습니다. Stochastic node가 Gumbel random variable로 넘어가면서 class probability $(\alpha_1, \alpha_2, \cdots, \alpha_K)$ 자체도 학습을 할 수 있게 됩니다.

지금까지 배운 내용을 토대로 dropout rate도 같이 학습할 수 있도록 실제 구현을 해보도록 하겠습니다. 들어가기에 앞서, 기본적인 컨셉은 다음과 같습니다.
* 원래 dropout neural network를 구현하는 데는 tf.nn.dropout(dropout_probability)와 같은 형태의 dropout layer를 기존의 layer 사이에 끼워넣으면 된다.
* **이를 dropout rate도 학습할 수 있는 concrete dropout layer를 만들어 기존의 dropout layer를 치환하도록 한다.**

먼저 필요한 package들을 import하도록 합니다.

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.python.layers import base
from tensorflow.python.layers import utils

from tensorflow.python.framework import tensor_shape
from tensorflow.python.ops import array_ops

그 다음 tf.nn.dropout처럼 layer 사이에 추가만 하면 되는 concrete dropout layer를 만들어보도록 하겠습니다.

In [3]:
class ConcreteDropout(base.Layer):
    """Concrete Dropout layer class from https://arxiv.org/abs/1705.07832.

    "Concrete Dropout" Yarin Gal, Jiri Hron, Alex Kendall

    Arguments:
        weight_regularizer:
            Positive float, satisfying $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$
            (inverse observation noise), and N the number of instances
            in the dataset.
        dropout_regularizer:
            Positive float, satisfying $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and
            N the number of instances in the dataset.
            The factor of two should be ignored for cross-entropy loss,
            and used only for the eucledian loss.
        init_min:
            Minimum value for the randomly initialized dropout rate, in [0, 1].
        init_min:
            Maximum value for the randomly initialized dropout rate, in [0, 1],
            with init_min <= init_max.
        name:
            String, name of the layer.
        reuse:
            Boolean, whether to reuse the weights of a previous layer
            by the same name.
    """

    def __init__(self, weight_regularizer=1e-6, dropout_regularizer=1e-5,
                 init_min=0.1, init_max=0.1, name=None, reuse=False,
                 training=True, **kwargs):

        super(ConcreteDropout, self).__init__(name=name, _reuse=reuse,
                                              **kwargs)
        assert init_min <= init_max, \
            'init_min must be lower or equal to init_max.'

        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        self.supports_masking = True
        self.p_logit = None
        self.p = None
        self.init_min = (np.log(init_min) - np.log(1. - init_min))
        self.init_max = (np.log(init_max) - np.log(1. - init_max))
        self.training = training
        self.reuse = reuse

    def get_kernel_regularizer(self):
        def kernel_regularizer(weight):
            if self.reuse:
                return None
            return self.weight_regularizer * tf.reduce_sum(tf.square(weight)) \
                / (1. - self.p)
        return kernel_regularizer

    def apply_dropout_regularizer(self, inputs):
        """
        위의 Evidence Lower BOund에서 H(p) = -p log p - (1 - p) log (1 - p)에 해당하는 부분입니다.
        """
        with tf.name_scope('dropout_regularizer'):
            input_dim = tf.cast(tf.reduce_prod(tf.shape(inputs)[1:]),
                                dtype=tf.float32)
            dropout_regularizer = self.p * tf.log(self.p)
            dropout_regularizer += (1. - self.p) * tf.log(1. - self.p)
            dropout_regularizer *= self.dropout_regularizer * input_dim
            tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES,
                                 dropout_regularizer)

    def build(self, input_shape):
        input_shape = tensor_shape.TensorShape(input_shape)
        self.input_spec = base.InputSpec(shape=input_shape)

        self.p_logit = self.add_variable(name='p_logit',
                                         shape=[],
                                         initializer=tf.random_uniform_initializer(
                                             self.init_min,
                                             self.init_max),
                                         dtype=tf.float32,
                                         trainable=True)
        self.p = tf.nn.sigmoid(self.p_logit, name='dropout_rate')
        tf.add_to_collection('DROPOUT_RATES', self.p)

        self.built = True

    def concrete_dropout(self, x):
        eps = 1e-7
        temp = 0.1

        """
        Gumbel softmax trick이 여기서 들어가게 됩니다.
        Bernoulli distribution의 continuous relaxation을 통해 sampling을 합니다.
        """
        with tf.name_scope('dropout_mask'):
            unif_noise = tf.random_uniform(shape=tf.shape(x))
            drop_prob = (
                tf.log(self.p + eps)
                - tf.log(1. - self.p + eps)
                + tf.log(unif_noise + eps)
                - tf.log(1. - unif_noise + eps)
            )
            drop_prob = tf.nn.sigmoid(drop_prob / temp)

        with tf.name_scope('drop'):
            random_tensor = 1. - drop_prob
            retain_prob = 1. - self.p
            x *= random_tensor
            x /= retain_prob

        return x

    def call(self, inputs, training=True):
        def dropped_inputs():
            return self.concrete_dropout(inputs)
        if not self.reuse:
            self.apply_dropout_regularizer(inputs)
        return utils.smart_cond(training,
                                dropped_inputs,
                                lambda: array_ops.identity(inputs))

Concrete dropout layer를 조금 더 편리하게 사용할 수 있도록 하는 functional interface를 만듭니다.

In [1]:
def concrete_dropout(inputs,
                     trainable=True,
                     weight_regularizer=1e-6,
                     dropout_regularizer=1e-5,
                     init_min=0.1, init_max=0.1,
                     training=True,
                     name=None,
                     reuse=False,
                     **kwargs):

    """Functional interface for Concrete Dropout.

    "Concrete Dropout" Yarin Gal, Jiri Hron, Alex Kendall
    from https://arxiv.org/abs/1705.07832.

    Arguments:
        weight_regularizer:
            Positive float, satisfying $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$
            (inverse observation noise), and N the number of instances
            in the dataset.
        dropout_regularizer:
            Positive float, satisfying $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and
            N the number of instances in the dataset.
            The factor of two should be ignored for cross-entropy loss,
            and used only for the eucledian loss.
        init_min:
            Minimum value for the randomly initialized dropout rate, in [0, 1].
        init_min:
            Maximum value for the randomly initialized dropout rate, in [0, 1],
            with init_min <= init_max.
        name:
            String, name of the layer.
        reuse:
            Boolean, whether to reuse the weights of a previous layer
            by the same name.

    Returns:
        Tupple containing:
            - the output of the dropout layer;
            - the kernel regularizer function for the subsequent
              convolutional layer.
    """

    layer = ConcreteDropout(weight_regularizer=weight_regularizer,
                            dropout_regularizer=dropout_regularizer,
                            init_min=init_min, init_max=init_max,
                            training=training,
                            trainable=trainable,
                            name=name,
                            reuse=reuse)
    return layer.apply(inputs, training=training), \
        layer.get_kernel_regularizer()

이제 실제 동작을 위한 모든 부분을 완성하였습니다. 앞서 언급했듯이, 기존의 dropout layer를 방금 완성한 concrete dropout layer로 대체하기만 하면 됩니다!! 

이제 MNIST dataset에 대해 dropout rate도 실제로 학습이 되는지 보도록 하겠습니다.
다음은 Layer를 위한 추가적인 module을 import하고 Convolutional neural network를 선언하는 부분입니다.

In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
def net(inputs, is_training):

    x = tf.reshape(inputs, [-1, 28, 28, 1])

    dropout_params = {'init_min': 0.1, 'init_max': 0.1,
                      'weight_regularizer': 1e-6, 'dropout_regularizer': 1e-5,
                      'training': is_training}
    x, reg = concrete_dropout(x, name='conv1_dropout', **dropout_params)
    x = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu, padding='SAME',
                   kernel_regularizer=reg, bias_regularizer=reg,
                   name='conv1')
    x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME', name='pool1')

    x, reg = concrete_dropout(x, name='conv2_dropout', **dropout_params)
    x = tf.layers.conv2d(x, 64, 5, activation=tf.nn.relu, padding='SAME',
                   kernel_regularizer=reg, bias_regularizer=reg,
                   name='conv2')
    x = tf.layers.max_pooling2d(x, 2, 2, padding='SAME', name='pool2')

    x = tf.reshape(x, [-1, 7*7*64], name='flatten')
    x, reg = concrete_dropout(x, name='fc1_dropout', **dropout_params)
    x = tf.layers.dense(x, 1024, activation=tf.nn.relu, name='fc1',
                  kernel_regularizer=reg, bias_regularizer=reg)

    outputs = tf.layers.dense(x, 10, name='fc2')
    return outputs

def net2(inputs, is_training):
    """
    위의 net 함수의 경우는 convolutional neural network를 나타냅니다.
    32 channels --> 64 channels --> 3136 (7*7*64) hidden units --> 1024 hidden units --> 10 units for class 0 ~ 9
    의 구조를 갖는 CNN architecture입니다.
    
    net2 함수에서는 hidden layer 2개를 갖는 fully connected models을 구현해보도록 합니다.
    두 개의 hidden layer는 각각 300개와 100개의 neuron을 가지도록 합니다.
    
    즉, 구현해야 할 fully connected models의 구조는
    784 input dimension --> 300 hidden units --> 100 hidden units --> 10 units for class 0 ~ 9
    입니다.
    """
    return

지금까지 MNIST 학습을 위한 모델을 만드는 코드였습니다. 이제 실제로 학습하는 code입니다.

In [8]:
def main(_):
    # MNIST dataset load
    mnist = input_data.read_data_sets('MNIST_data')

    # MNIST has 28x28 = 784 input dimension and 10 classes
    x = tf.placeholder(tf.float32, [None, 784])
    y = tf.placeholder(tf.int64, [None])
    is_training = tf.placeholder(tf.bool)

    y_out = net(x, is_training)
    """
    y_out의 경우 현재 CNN을 통해 predict된 label을 의미합니다.
    위의 net2로 구현한 Fully-connected model에 대해서도 확인해보도록 합니다.
    """

    # softmax loss function
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(
                labels=y, logits=y_out))
        loss += tf.reduce_sum(
                tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))

    # optimizer we will use
    with tf.name_scope('adam_optimizer'):
        train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

    # to keep track of model accuracy
    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(y_out, 1), y)
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)

    # dropout rates for all layers
    dropout_rates = tf.get_collection('DROPOUT_RATES')
    def rates_pretty_print(values):
        return {str(t.name): round(r, 4)
                for t, r in zip(dropout_rates, values)}

    # actual running part!!
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # How many iterations?
        for i in range(5000):
            # Mini-batch size!
            batch = mnist.train.next_batch(50)
            if i % 500 == 0:
                training_loss, training_acc, rates = sess.run(
                        [loss, accuracy, dropout_rates],
                        feed_dict={
                            x: batch[0], y: batch[1], is_training: False})
                print('step {}, loss {}, accuracy {}'.format(
                    i, training_loss, training_acc))
                print('dropout rates: {}'.format(rates_pretty_print(rates)))
            train_step.run(feed_dict={
                x: batch[0], y: batch[1], is_training: True})

        accuracy, rates = sess.run([accuracy, dropout_rates],
                                   feed_dict={x: mnist.test.images,
                                              y: mnist.test.labels,
                                              is_training: False})
        print('test accuracy {}'.format(accuracy))
        print('final dropout rates: {}'.format(rates_pretty_print(rates)))

In [ ]:
tf.app.run(main=main)

실제로 dropout rate가 각 layer별로 다르게 학습되는 것을 볼 수 있습니다.

### 연습 문제 (Hint는 코드 및 코드의 주석을 참고해주세요!)

**Q1.** iteration 횟수를 10000번, 20000번, 40000번으로 나눠서 학습을 진행해보세요.

**Q2.** mini-batch size를 100, 200에 대해서 학습을 진행해보세요.

**Q3 (Miscellaneous).** 지금은 convolutional layer에 대해서 학습을 해보았는데 hidden layer가 2개이며 각각이 300개와 100개의 unit을 갖는 fully-connected model에 대해서 학습을 진행해보세요 (즉, 784 (MNIST input dimension) - 300 - 100 - 10 (output dimension)의 structure를 갖는 모델입니다). --> 함수 net2 부분을 구현해주세요.